# Capstone Project Battle of Neighbourhood - Week 5
## Haitham Mostafa

## Problem and a discussion of the background

Since London UK is one of the most expensive places to live in, the project is established in order to help the property seekers in order to get the proper recommendation based on their budget, using existing open source data officially provided by the UK government, data science can be used in order to provide the recommendations together with the most popular venues related to this recommendation.

The project is generic and can be used for any place with available data for properties purchase transactions.

## Description of the data and how it will be used to solve the problem

Data provided by UK government in the form of CSV file with all the transactions for property purchasing with London and Wales, can be used in order to filter the proper districts for each property seekers budget and accordingly they can get all related facilities and point of interest using foursquare API.

Data Set Source: http://landregistry.data.gov.uk/


In [10]:
import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [11]:
# Read the data for examination (Source: http://landregistry.data.gov.uk/)
df_ppd = pd.read_csv("PPMON.csv")

# Assign meaningful column names
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [12]:
# Format the date column
df_ppd['Date_Transfer'] = df_ppd['Date_Transfer'].apply(pd.to_datetime)

# Delete all obsolete transactions which were done before 2016
df_ppd.drop(df_ppd[df_ppd.Date_Transfer.dt.year < 2016].index, inplace=True)

# Sort by Date of Sale
df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

In [13]:
df_ppd_london = df_ppd.query("Town_City == 'LONDON'")

# Make a list of street names in LONDON
streets = df_ppd_london['Street'].unique().tolist()

In [14]:
# Average Price per Street
df_grp_price = df_ppd_london.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
df_grp_price.columns = ['Street', 'Avg_Price']

In [15]:
# Place the required Budget
df_affordable = df_grp_price.query("(Avg_Price >= 2100000) & (Avg_Price <= 2400000)")

In [16]:
df_affordable

,Street,Avg_Price
71,ALMEIDA STREET,2.302000e+06
151,BALHAM HIGH ROAD,2.183333e+06
152,BALHAM HILL,2.375000e+06
156,BALLINGDON ROAD,2.365000e+06
265,BICKENHALL STREET,2.300000e+06
342,BRAMPTON GROVE,2.400000e+06
393,BRONDESBURY PARK,2.400000e+06
620,CHILTERN STREET,2.225000e+06
631,CHIVALRY ROAD,2.100000e+06
772,CORNWALL GARDENS,2.300000e+06


In [17]:
i=0

for i, row in df_affordable.iterrows():
    #print(df_affordable.loc[i,"Street"])
    #print(i)
    geolocator = Nominatim()
    location = geolocator.geocode(df_affordable.loc[i,"Street"] , timeout= None)
    latitude = location.latitude
    longitude = location.longitude
    #df_affordable['latitude']= latitude
    #df_affordable['longitude']= longitude
    df_affordable.loc[i,"latitude"]= latitude
    df_affordable.loc[i,"longitude"]= longitude
#done
df_affordable.head(10)

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  
/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: `timeout=None` has been passed to a geocoder call. Using default geocoder timeout. In geopy 2.0 the behavior will be different: None will mean "no timeout" instead of "default geocoder timeout". Pass geopy.geocoders.base.DEFAULT_SENTINEL instead of None to get rid of this warning.
  import sys


,Street,Avg_Price,latitude,longitude
71,ALMEIDA STREET,2.302000e+06,51.539575,-0.103625
151,BALHAM HIGH ROAD,2.183333e+06,51.447621,-0.148760
152,BALHAM HILL,2.375000e+06,51.452020,-0.147747
156,BALLINGDON ROAD,2.365000e+06,51.454189,-0.158856
265,BICKENHALL STREET,2.300000e+06,51.521197,-0.158934
342,BRAMPTON GROVE,2.400000e+06,51.570365,-0.283394
393,BRONDESBURY PARK,2.400000e+06,51.540439,-0.210419
620,CHILTERN STREET,2.225000e+06,51.521626,-0.156205
631,CHIVALRY ROAD,2.100000e+06,51.458021,-0.171349
772,CORNWALL GARDENS,2.300000e+06,18.466400,-77.911074


In [28]:
address = 'London, UK'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df_affordable['latitude'], df_affordable['longitude'], df_affordable['Avg_Price'], df_affordable['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of London City are 51.5073219, -0.1276474.


In [29]:
CLIENT_ID = 'PBN2SJQCSJWQ4LZESXKUWA3JBBKFVS34YPLDGOKSWDSJXXLB' # your Foursquare ID
CLIENT_SECRET = 'ETJDG1DMFEP3OOCBCLMBNJ2SQC1L3BDVQKNON0JY4B33MXCG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PBN2SJQCSJWQ4LZESXKUWA3JBBKFVS34YPLDGOKSWDSJXXLB
CLIENT_SECRET:ETJDG1DMFEP3OOCBCLMBNJ2SQC1L3BDVQKNON0JY4B33MXCG


In [30]:
#Prefered locations venues

def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
location_venues = getNearbyVenues(names=df_affordable['Street'],
                                   latitudes=df_affordable['latitude'],
                                   longitudes=df_affordable['longitude']
                                  )

ALMEIDA STREET
BALHAM HIGH ROAD
BALHAM HILL
BALLINGDON ROAD
BICKENHALL STREET
BRAMPTON GROVE
BRONDESBURY PARK
CHILTERN STREET
CHIVALRY ROAD
CORNWALL GARDENS
DALEHAM MEWS
EARLS COURT SQUARE
FLORENCE STREET
GRAND AVENUE
JUBILEE CLOSE
KANGLEY BRIDGE ROAD
LEINSTER MEWS
LILYVILLE ROAD
MANSON MEWS
MARLBOROUGH CRESCENT
MOORGATE
OFFERTON ROAD
PEMBRIDGE MEWS
PLATTS LANE
PORTOBELLO ROAD
ROWAN ROAD
SKEFFINGTON STREET
ST MAUR ROAD
STRAND
SUFFOLK ROAD
THREE COLT STREET
TRINITY CRESCENT
WESTMORELAND PLACE
WINGATE ROAD
WINTERBROOK ROAD
WOODVILLE GARDENS
YOUNG STREET


In [33]:
location_venues

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ALMEIDA STREET,51.539575,-0.103625,Ottolenghi,51.539716,-0.102314,Mediterranean Restaurant
1,ALMEIDA STREET,51.539575,-0.103625,Almeida Theatre,51.539620,-0.102845,Theater
2,ALMEIDA STREET,51.539575,-0.103625,The Taproom,51.540782,-0.102704,Pub
3,ALMEIDA STREET,51.539575,-0.103625,MEAT Liquor,51.539154,-0.102844,Burger Joint
4,ALMEIDA STREET,51.539575,-0.103625,Little Angel Theatre,51.538759,-0.101295,Theater
5,ALMEIDA STREET,51.539575,-0.103625,Zaffrani's,51.539255,-0.101202,Indian Restaurant
6,ALMEIDA STREET,51.539575,-0.103625,Amorino,51.538034,-0.102583,Ice Cream Shop
7,ALMEIDA STREET,51.539575,-0.103625,King's Head Theatre Pub,51.538249,-0.102404,Pub
8,ALMEIDA STREET,51.539575,-0.103625,The Regent,51.538952,-0.106809,Pub
9,ALMEIDA STREET,51.539575,-0.103625,Le Mercury,51.539697,-0.102715,French Restaurant


In [34]:
location_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
ALMEIDA STREET,98,98,98,98,98,98
BALHAM HIGH ROAD,38,38,38,38,38,38
BALHAM HILL,25,25,25,25,25,25
BALLINGDON ROAD,11,11,11,11,11,11
BICKENHALL STREET,95,95,95,95,95,95
BRAMPTON GROVE,4,4,4,4,4,4
BRONDESBURY PARK,9,9,9,9,9,9
CHILTERN STREET,100,100,100,100,100,100
CHIVALRY ROAD,46,46,46,46,46,46


In [35]:
#1st Neighbourhood
def getNearbyFacility(names, latitudes, longitudes, radius=5000):
    
    facility_list=[]
    
    radius = 5000
    LIMIT = 100
    categories = '4bf58dd8d48988d196941735,58daa1558bbb0b01f18ec1f7,4bf58dd8d48988d13d941735,4f4533804b9074f6e4fb0105,4bf58dd8d48988d118951735'
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&categoryId={}&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        categories,
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT)

        # make the GET request
        results = requests.get(url).json()['response']

        # return only relevant information for each nearby venue
        facility_list.append([(
            name,
            facility['name'],
            facility['categories'][0]['name'],
            facility['location']['distance'], 
            facility['location']['lat'],
            facility['location']['lng']) for facility in results['venues']])

    nearby_facility = pd.DataFrame([item for f_list in facility_list for item in f_list])
    nearby_facility.columns = ['Street Name',
                  'Facility Name',             
                  'Facility Category', 
                  'Distance', 
                  'Facility Latitude', 
                  'Facility Longitude']
    
    return(nearby_facility)

In [38]:
location_facility = getNearbyFacility(names=df_affordable['Street'],
                                      latitudes=df_affordable['latitude'],
                                      longitudes=df_affordable['longitude']
                                  )

ALMEIDA STREET
BALHAM HIGH ROAD
BALHAM HILL
BALLINGDON ROAD
BICKENHALL STREET
BRAMPTON GROVE
BRONDESBURY PARK
CHILTERN STREET
CHIVALRY ROAD
CORNWALL GARDENS
DALEHAM MEWS
EARLS COURT SQUARE
FLORENCE STREET
GRAND AVENUE
JUBILEE CLOSE
KANGLEY BRIDGE ROAD
LEINSTER MEWS
LILYVILLE ROAD
MANSON MEWS
MARLBOROUGH CRESCENT
MOORGATE
OFFERTON ROAD
PEMBRIDGE MEWS
PLATTS LANE
PORTOBELLO ROAD
ROWAN ROAD
SKEFFINGTON STREET
ST MAUR ROAD
STRAND
SUFFOLK ROAD
THREE COLT STREET
TRINITY CRESCENT
WESTMORELAND PLACE
WINGATE ROAD
WINTERBROOK ROAD
WOODVILLE GARDENS
YOUNG STREET


In [39]:
location_facility

,Street Name,Facility Name,Facility Category,Distance,Facility Latitude,Facility Longitude
0,ALMEIDA STREET,Tesco,Grocery Store,405,51.536001,-0.102466
1,ALMEIDA STREET,Fortnum & Mason,Department Store,4238,51.508363,-0.138667
2,ALMEIDA STREET,Tesco,Grocery Store,5220,51.518411,-0.170889
3,ALMEIDA STREET,Marks & Spencer,Department Store,4503,51.514064,-0.154100
4,ALMEIDA STREET,Tesco Metro,Grocery Store,4202,51.504308,-0.081982
5,ALMEIDA STREET,Royal Free Hospital,Hospital,4601,51.553714,-0.166091
6,ALMEIDA STREET,Tesco Metro,Grocery Store,3237,51.526906,-0.061534
7,ALMEIDA STREET,Whole Foods Market,Grocery Store,3188,51.562138,-0.075250
8,ALMEIDA STREET,The London Clinic,Hospital,3741,51.522925,-0.150562
9,ALMEIDA STREET,Japan Centre,Japanese Restaurant,3831,51.509761,-0.131273


In [40]:
location_facility.groupby('Street Name').count()

,Facility Name,Facility Category,Distance,Facility Latitude,Facility Longitude
Street Name,,,,,
ALMEIDA STREET,50,50,50,50,50
BALHAM HIGH ROAD,50,50,50,50,50
BALHAM HILL,50,50,50,50,50
BALLINGDON ROAD,50,50,50,50,50
BICKENHALL STREET,50,50,50,50,50
BRAMPTON GROVE,50,50,50,50,50
BRONDESBURY PARK,50,50,50,50,50
CHILTERN STREET,50,50,50,50,50
CHIVALRY ROAD,50,50,50,50,50
